# Source detction

In [49]:
import numpy as np
import matplotlib
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
from astroquery.sdss import SDSS
from astropy import units as u
from astropy.nddata import Cutout2D

In [50]:
from astropy.io import fits
from astropy import wcs
fits_image_filename = '../Data/newfits/calexp-1-108438.fits'

hdu_list = fits.open(fits_image_filename)
hdu = fits.open(fits_image_filename)[1]
w = wcs.WCS(hdu.header)

In [51]:
image_data = fits.getdata(fits_image_filename,0)
print(type(image_data))
print(image_data.shape)
# Parse the WCS keywords in the primary HDU
print(w)

<class 'numpy.ndarray'>
(6132, 8176)
WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN-SIP'  'DEC--TAN-SIP'  
CRVAL : 151.891410835415  41.777202445749  
CRPIX : 4074.320544  3065.826822  
CD1_1 CD1_2  : -9.05112531824021e-07  -0.000347023189685797  
CD2_1 CD2_2  : 0.000346982413101619  -9.20137431574141e-07  
NAXIS : 8176  6132


In [52]:
from astropy.convolution import Gaussian2DKernel
from photutils import Background2D, MedianBackground
from photutils import detect_threshold, detect_sources
data = image_data
bkg_estimator = MedianBackground()
bkg = Background2D(data, (50, 50), filter_size=(3, 3), bkg_estimator=bkg_estimator)
threshold = bkg.background + (10. * bkg.background_rms)

In [53]:
from astropy.stats import gaussian_fwhm_to_sigma
sigma = 3.0 * gaussian_fwhm_to_sigma  # FWHM = 3.
kernel = Gaussian2DKernel(sigma, x_size=3, y_size=3)
kernel.normalize()
npixels = 5
segm = detect_sources(data, threshold, npixels=npixels,filter_kernel=kernel)
#segm_deblend = deblend_sources(data, segm, npixels=npixels,filter_kernel=kernel, nlevels=32,contrast=0.001)

In [54]:
from photutils import source_properties
cat = source_properties(data, segm, wcs=w)
tbl = cat.to_table()
df = tbl.to_pandas()

In [55]:
indexNames = df[df['xcentroid'] < 400].index
dfsel = df.drop(indexNames)
indexNames = dfsel[dfsel['xcentroid'] > 7776].index
dfsel = dfsel.drop(indexNames)
indexNames = dfsel[dfsel['ycentroid'] < 300].index
dfsel = dfsel.drop(indexNames)
indexNames = dfsel[dfsel['ycentroid'] > 5832].index
dfsel = dfsel.drop(indexNames)
indexNames = dfsel[dfsel['elongation'] > 10].index
dfsel = dfsel.drop(indexNames)
print(dfsel)
dfsel = dfsel[['xcentroid','ycentroid','sky_centroid.ra','sky_centroid.dec','elongation','equivalent_radius','area']]

        id    xcentroid    ycentroid  sky_centroid.ra  sky_centroid.dec  \
117    118  1751.308771   302.236109       153.158214         40.970579   
118    119  6276.520949   309.069830       153.181520         42.533604   
119    120  3010.498063   311.189794       153.162926         41.404998   
120    121   989.355775   314.224136       153.146903         40.708890   
121    122  3169.523926   323.080086       153.158494         41.460012   
...    ...          ...          ...              ...               ...   
2755  2756  1463.793207  5826.794206       150.633059         40.866663   
2756  2757  1596.409726  5828.523477       150.630972         40.912242   
2757  2758  2545.994328  5827.253490       150.622545         41.239467   
2758  2759  5713.886492  5831.000413       150.595120         42.334878   
2760  2761  2262.001952  5830.381343       150.623752         41.141460   

      sky_centroid_icrs.ra  sky_centroid_icrs.dec     source_sum  \
117             153.158214     

In [56]:
coords = list(zip(dfsel.xcentroid, dfsel.ycentroid))
print(coords)
dfsel['coords'] = coords

[(1751.3087707300392, 302.23610873653615), (6276.520948560798, 309.06982992010893), (3010.498063471877, 311.1897942077577), (989.3557754598116, 314.224136474167), (3169.5239258089796, 323.0800858671331), (5755.396830748483, 321.13182737693865), (7483.153883766415, 331.16013551271305), (2914.7553524374175, 333.9852602108037), (5920.2296063463555, 348.3383226063963), (417.86287436428705, 349.41208325485024), (2990.312242090784, 350.3370013755158), (2285.6273344651954, 360.1281833616299), (3855.6561570596023, 366.5308067871203), (7660.924695134728, 368.5958596991158), (956.0865234982882, 366.8502956738251), (2908.9823549372245, 366.99864268747876), (3026.401148390002, 370.3285014636343), (6390.762809187279, 370.879785041225), (4748.976712140003, 376.38941711451014), (5157.875801472334, 376.6652814058418), (7472.313293970299, 380.9794238683128), (960.8990505359877, 381.72551301684535), (7344.146007789679, 383.24333008763386), (1139.1108916478556, 382.61258465011286), (3089.5608416725854, 4

# Cross Referencing

In [57]:
lon1, lat1 = w.all_pix2world(0, 0, 0)
lon2, lat2 = w.all_pix2world(6132, 8176, 0)
maxlon = max(lon1,lon2)
minlon = min(lon1,lon2)
maxlat = max(lat1,lat2)
minlat = min(lat1,lat2)

In [58]:
query = f"""
SELECT ra,dec
FROM Galaxy
WHERE ra between {minlon} and {maxlon}
AND dec between {minlat} and {maxlat}
AND g < 21
"""
res = SDSS.query_sql(query)
galaxies = res.to_pandas()

In [59]:
print(len(galaxies))

7037


In [60]:
query = f"""
SELECT ra,dec
FROM Star
WHERE ra between {minlon} and {maxlon}
AND dec between {minlat} and {maxlat}
AND g < 21
"""
res = SDSS.query_sql(query)
stars = res.to_pandas()

In [61]:
print(len(stars))

9121


In [63]:
print(galaxies)

              ra        dec
0     149.592670  40.860024
1     149.592689  41.302531
2     149.593090  41.552851
3     149.594043  40.810266
4     149.699064  42.303802
...          ...        ...
7032  153.275935  40.734086
7033  153.275983  41.879107
7034  153.276057  41.459061
7035  153.276376  42.319513
7036  153.276701  41.389201

[7037 rows x 2 columns]


In [64]:
from astropy.coordinates import SkyCoord
from astropy import units as u
c = SkyCoord(ra=dfsel['sky_centroid.ra']*u.degree, dec=dfsel['sky_centroid.dec']*u.degree)
catalog = SkyCoord(ra=galaxies['ra']*u.degree, dec=galaxies['dec']*u.degree)

max_sep = 3.0 * u.arcsec
idx, d2d, d3d = c.match_to_catalog_3d(catalog)
sep_constraint = d2d < max_sep
c_matches = c[sep_constraint]
catalog_matches = catalog[idx[sep_constraint]]

In [66]:
print(c_matches)

<SkyCoord (ICRS): (ra, dec) in deg
    [(153.11991272, 41.43265675), (152.98825898, 42.3854919 ),
     (152.9690753 , 42.16505681), (152.94339375, 41.28434169),
     (152.93832323, 42.36679847), (152.90194137, 41.41697205),
     (152.88585985, 41.73150125), (152.79475761, 41.40742319),
     (152.77503641, 40.61835572), (152.76682358, 42.04630853),
     (152.6328261 , 41.50733112), (152.62109244, 41.60642747),
     (152.6165157 , 42.26391727), (152.55040738, 41.06283394),
     (152.37585706, 41.44029872), (152.35647744, 40.92113578),
     (152.31035051, 40.67668792), (152.30610859, 40.89105877),
     (152.15759714, 42.10984932), (152.06808804, 40.80405639),
     (152.05740796, 41.90812787), (152.05447637, 40.86540347),
     (152.04806533, 42.4186249 ), (152.04537277, 41.21871943),
     (152.03141702, 40.6500885 ), (152.03069298, 40.87180083),
     (152.02545177, 40.78855294), (152.0006072 , 42.42282711),
     (151.86543479, 41.13905559), (151.77419806, 42.18298148),
     (151.76756569, 

In [70]:
print(len(c_matches))

75


# Cutting out

In [ ]:
size = u.Quantity([40, 40], u.pixel)
for i in range(len(c_matches)):
    # Make the cutout, including the WCS
    cutout = Cutout2D(image_data, position=c_matches[i], size=size, wcs=w)

    # Put the cutout image in the FITS HDU
    hdu = fits.PrimaryHDU(cutout.data)

    # Update the FITS header with the cutout WCS
    hdu.header.update(cutout.wcs.to_header(relax=False))


    # Write the cutout to a new FITS file
    cutout_filename = '006.fits'
    hdu.writeto(cutout_filename, overwrite=True)

In [68]:
size = u.Quantity([40, 40], u.pixel)

# Make the cutout, including the WCS
cutout = Cutout2D(image_data, position=c_matches[2], size=size, wcs=w)

# Put the cutout image in the FITS HDU
hdu = fits.PrimaryHDU(cutout.data)

# Update the FITS header with the cutout WCS
hdu.header.update(cutout.wcs.to_header(relax=False))


# Write the cutout to a new FITS file
cutout_filename = '006.fits'
hdu.writeto(cutout_filename, overwrite=True)

In [30]:
size = u.Quantity([40, 40], u.pixel)

# Make the cutout, including the WCS
cutout = Cutout2D(image_data, position=coords[1000], size=size, wcs=w)

# Put the cutout image in the FITS HDU
hdu = fits.PrimaryHDU(cutout.data)

# Update the FITS header with the cutout WCS
hdu.header.update(cutout.wcs.to_header(relax=False))


# Write the cutout to a new FITS file
cutout_filename = '004.fits'
hdu.writeto(cutout_filename, overwrite=True)

NameError: name 'Cutout2D' is not defined